In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lecture 1: Reading Image Data from Files

In [ ]:
image_shape = (300, 300)
image_path = '/content/drive/MyDrive/garbage_classification'

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(image_path, shuffle=True, batch_size=32,
                                                            image_size=image_shape, subset='training',
                                                            validation_split=0.3, seed=42)
# seed = what pattern the random shuffle follows
# Must have same seeds, b/c the seed determines what the data is testing and validating

validation_dataset = tf.keras.utils.image_dataset_from_directory(image_path, shuffle=True, batch_size=32,
                                                                image_size=image_shape, subset='training',
                                                                validation_split=0.3, seed=42)

# This reduces the run time by reducing the data (only do this for practice not for real)
train_batches = tf.data.experimental.cardinality(train_dataset)
train_dataset = train_dataset.take(train_batches // 10) # // means for every fifth piece of data, for every 5 batches, take one of them (reduces to 20%)

# Establish testing data
validation_batches = tf.data.experimental.cardinality(validation_dataset)
validation_dataset = validation_dataset.skip(train_batches // 5) # // skip every fifth one (gets 80% of original data)
test_dataset = validation_dataset.take(validation_batches // 5)

# We want to gather as many images as we can based on RAM amount before training. The amount we can gather at one time is limited by RAM.
# AUTOTUNE helps us figure out how much RAM we have
# buffer_size is just how many images we can import, which is determined by AUTOTUNE
# Once one batch is through, we will go to the next batch, and then the next batch
# Let hardware determine space for preloading images

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 15515 files belonging to 12 classes.
Using 10861 files for training.
Found 15515 files belonging to 12 classes.
Using 10861 files for training.


In [ ]:
image_final_shape = image_shape + (3,) # We need to add our channels
num_classes = 12 # 12 classes
print(image_final_shape)

(300, 300, 3)


# Lecture 2: Implement the Convolutional Network

My Notes:

MaxPooling takes the "footprint", it reduces dimensions by taking a value from the quadrants (see slides)

Global Average does the same thign as MaxPooling but takes the average

In [ ]:
### New Code Here! ###
data_augmentation = tf.keras.Sequential([tf.keras.layers.Rescaling(1/255),
                                         tf.keras.layers.RandomFlip('horizontal'),
                                         tf.keras.layers.RandomRotation(0.2)])

### End New Code ###

prediction_layer = tf.keras.layers.Dense(12, activation='softmax')
inputs = tf.keras.Input(shape=image_final_shape)
### New Code Here! ###
x = data_augmentation(inputs)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3))(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x) # Image is shrunk in half
# The number in the dimension of a square pool_size is the denominator for how much image shrinks
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3))(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3))(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Flatten()(x)
# x = tf.keras.layers.Dense(96)(x)

### End New Code ###
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.001

model_optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model_loss=tf.keras.losses.SparseCategoricalCrossentropy()
model_accuracy_metric=tf.keras.metrics.SparseCategoricalAccuracy(name='sparse_categorical_accuracy')

model.compile(optimizer=model_optimizer, loss=model_loss,
              metrics=[model_accuracy_metric])

model_epochs = 10

model_early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,
                                                       restore_best_weights=True,
                                                       monitor='val_loss')

model_history = model.fit(train_dataset, epochs=model_epochs,
                         validation_data=validation_dataset,
                         callbacks=[model_early_stopping])

model_accuracy = model_history.history['sparse_categorical_accuracy']
model_val_accuracy = model_history.history['val_sparse_categorical_accuracy']

model_loss = model_history.history['loss']
model_val_loss = model_history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(model_accuracy, label='Training Accuracy')
plt.plot(model_val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 2)
plt.plot(model_loss, label='Training Loss')
plt.plot(model_val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.show()